In [19]:
import requests
import bs4
from bs4 import BeautifulSoup 

## Downloading weather data

page = requests.get("https://forecast.weather.gov/MapClick.php?lat=34.03707000000003&lon=-118.26264499999996#.YI534y9h02I")
soup = BeautifulSoup(page.content,"html.parser")
seven_day = soup.find(id ="seven-day-forecast")                  
forecast_class = seven_day.find_all(class_= "tombstone-container") 
tonight = forecast_class[0]
print(tonight.prettify())

<div class="tombstone-container">
 <p class="period-name">
  Overnight
  <br/>
  <br/>
 </p>
 <p>
  <img alt="Overnight: Patchy fog.  Otherwise, cloudy, with a low around 60. Calm wind. " class="forecast-icon" src="newimages/medium/nfg.png" title="Overnight: Patchy fog.  Otherwise, cloudy, with a low around 60. Calm wind. "/>
 </p>
 <p class="short-desc">
  Patchy Fog
 </p>
 <p class="temp temp-low">
  Low: 60 °F
 </p>
</div>


In [20]:
## Extracting information from the page

period = tonight.find(class_="period-name").get_text()
short_desc = tonight.find(class_="short-desc").get_text()
temp = tonight.find(class_="temp").get_text()
print(period)
print(short_desc)
print(temp)

Overnight
Patchy Fog
Low: 60 °F


In [21]:
img = tonight.find("img")
tonight_title = img['title']
print(tonight_title)

Overnight: Patchy fog.  Otherwise, cloudy, with a low around 60. Calm wind. 


In [22]:
## Extracting all the information from the page
all_period_names = seven_day.select(".tombstone-container .period-name")
all_period_name =[i.get_text() for i in all_period_names]
all_period_name

['Overnight',
 'Sunday',
 'SundayNight',
 'Monday',
 'MondayNight',
 'Tuesday',
 'TuesdayNight',
 'Wednesday',
 'WednesdayNight']

In [23]:
all_period_descs = seven_day.select(".tombstone-container .short-desc")
all_period_desc = [i.get_text() for i in all_period_descs]
all_period_desc

['Patchy Fog',
 'Patchy Fogthen Sunny',
 'Patchy Fog',
 'Patchy Fogthen Sunny',
 'Patchy Fog',
 'Patchy Fogthen Sunny',
 'Mostly Clear',
 'Sunny',
 'Patchy Fog']

In [24]:
all_period_temps = seven_day.select(".forecast-tombstone .temp")
all_period_temp =[i.get_text() for i in all_period_temps]
all_period_temp

['Low: 60 °F',
 'High: 71 °F',
 'Low: 59 °F',
 'High: 78 °F',
 'Low: 59 °F',
 'High: 82 °F',
 'Low: 61 °F',
 'High: 85 °F',
 'Low: 60 °F']

In [25]:
all_period_titles = seven_day.select(".forecast-tombstone img")
all_period_title = [i["title"] for i in all_period_titles]
all_period_title

['Overnight: Patchy fog.  Otherwise, cloudy, with a low around 60. Calm wind. ',
 'Sunday: Patchy fog before 11am.  Otherwise, cloudy through mid morning, then gradual clearing, with a high near 71. Light and variable wind becoming southwest 5 to 10 mph in the morning. ',
 'Sunday Night: Patchy fog after 11pm.  Otherwise, increasing clouds, with a low around 59. Southwest wind 5 to 10 mph becoming light and variable. ',
 'Monday: Patchy fog before 11am.  Otherwise, cloudy through mid morning, then gradual clearing, with a high near 78. Light and variable wind becoming west southwest 5 to 10 mph in the afternoon. ',
 'Monday Night: Patchy fog after 11pm.  Otherwise, mostly cloudy, with a low around 59. West southwest wind 5 to 10 mph becoming light and variable  in the evening. ',
 'Tuesday: Patchy fog before 11am.  Otherwise, mostly sunny, with a high near 82.',
 'Tuesday Night: Mostly clear, with a low around 61.',
 'Wednesday: Sunny, with a high near 85.',
 'Wednesday Night: Patchy f

In [26]:
## Combining our data into a Pandas Dataframe
import pandas
weather = pandas.DataFrame({
    "period":all_period_name,
    "short_desc":all_period_desc,
    "temp":all_period_temp,
    "desc":all_period_desc
})

weather

,period,short_desc,temp,desc
0,Overnight,Patchy Fog,Low: 60 °F,Patchy Fog
1,Sunday,Patchy Fogthen Sunny,High: 71 °F,Patchy Fogthen Sunny
2,SundayNight,Patchy Fog,Low: 59 °F,Patchy Fog
3,Monday,Patchy Fogthen Sunny,High: 78 °F,Patchy Fogthen Sunny
4,MondayNight,Patchy Fog,Low: 59 °F,Patchy Fog
5,Tuesday,Patchy Fogthen Sunny,High: 82 °F,Patchy Fogthen Sunny
6,TuesdayNight,Mostly Clear,Low: 61 °F,Mostly Clear
7,Wednesday,Sunny,High: 85 °F,Sunny
8,WednesdayNight,Patchy Fog,Low: 60 °F,Patchy Fog


In [27]:
temp_numbers = weather["temp"].str[-5:-2]
weather["temp_numbers"] = temp_numbers.astype('int')
temp_numbers

0    60 
1    71 
2    59 
3    78 
4    59 
5    82 
6    61 
7    85 
8    60 
Name: temp, dtype: object

In [28]:
weather["temp_numbers"].mean()

68.33333333333333

In [29]:
is_night = weather["temp"].str.contains("Low")
weather["is_night"]= is_night
is_night

0     True
1    False
2     True
3    False
4     True
5    False
6     True
7    False
8     True
Name: temp, dtype: bool

In [30]:
weather[is_night]

,period,short_desc,temp,desc,temp_numbers,is_night
0,Overnight,Patchy Fog,Low: 60 °F,Patchy Fog,60,True
2,SundayNight,Patchy Fog,Low: 59 °F,Patchy Fog,59,True
4,MondayNight,Patchy Fog,Low: 59 °F,Patchy Fog,59,True
6,TuesdayNight,Mostly Clear,Low: 61 °F,Mostly Clear,61,True
8,WednesdayNight,Patchy Fog,Low: 60 °F,Patchy Fog,60,True
